In [1]:
import pandas as pd

import pickle

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

nltk.download('punkt')
nltk.download('stopwords')
stop = set(stopwords.words('english'))

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

def tokenizer_lemmatizer(text):
    
    # tokenizes, removes, stop words, gets pos, lemmatizes with pos
    tokenized_text = word_tokenize(text)
    
    no_stops = []
    for word in tokenized_text:
        if word not in stop:
            no_stops.append(word)
    tokenized_text = no_stops
    
    pos_ = {"N": wordnet.NOUN,
            "V": wordnet.VERB,
            "J": wordnet.ADJ,
            "R": wordnet.ADV}
    lemmatized = []
    tags = nltk.pos_tag(tokenized_text)
    for word in tags:
        tag = word[1][0]
        if tag not in pos_: # anything not in .lemmatize()'s pos dictionary is considered a noun
            tag = 'N'
        tag = pos_[tag]
        lem = lemmatizer.lemmatize(word[0], tag)
        lemmatized.append(lem)
    
    return lemmatized

def preprocessor_remove_punc(text):
    return text.lower().translate(str.maketrans('', '', string.punctuation))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Spencer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Spencer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Spencer\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
kr_best_classifier = pickle.load(open("kr_best_estimator.sav", "rb"))

C:\Users\Spencer\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Spencer\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Spencer\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Spencer\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LinearSVC from version 0.21.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use a

In [19]:
kr_df = pd.read_csv("scrape/r_kamenrider_comments_cut.csv")

In [20]:
kr_df.head()

,id,_reddit,_replies,_submission,author,body,controversiality,created,created_utc,depth,distinguished,gilded,is_submitter,link_id,no_follow,parent_id,permalink,score,score_hidden,send_replies
0,c1ze8ay,<praw.reddit.Reddit object at 0x0000022989DB9710>,<praw.models.comment_forest.CommentForest obje...,hy74u,Sparrowsluck,Oh man suddenly there are people here. I shoul...,0,1.307984e+09,1.307955e+09,0,NaN,0,False,t3_hy74u,True,t3_hy74u,/r/KamenRider/comments/hy74u/this_falls_kamen_...,2,False,True
1,c205hdu,<praw.reddit.Reddit object at 0x0000022989DB9710>,<praw.models.comment_forest.CommentForest obje...,i1nme,StimulusResponse,This was a great episode. I'm not going to spo...,0,1.308295e+09,1.308267e+09,0,NaN,0,True,t3_i1nme,True,t3_i1nme,/r/KamenRider/comments/i1nme/ooo_38_hd/c205hdu/,1,False,True
2,c25yena,<praw.reddit.Reddit object at 0x0000022989DB9710>,<praw.models.comment_forest.CommentForest obje...,igxfl,Sparrowsluck,"I got it started, abandoned it for a while, an...",0,1.310818e+09,1.310790e+09,0,NaN,0,False,t3_igxfl,True,t3_igxfl,/r/KamenRider/comments/igxfl/this_is_where_i_n...,2,False,True
3,c260el5,<praw.reddit.Reddit object at 0x0000022989DB9710>,<praw.models.comment_forest.CommentForest obje...,ir1ai,tinkyXIII,One more episode? So they're cutting the seaso...,0,1.310858e+09,1.310830e+09,0,NaN,0,False,t3_ir1ai,True,t3_ir1ai,/r/KamenRider/comments/ir1ai/kamen_rider_ooo_4...,1,False,True
4,c261pqw,<praw.reddit.Reddit object at 0x0000022989DB9710>,<praw.models.comment_forest.CommentForest obje...,ir1ai,StimulusResponse,Good think wherever you heard that was dead wr...,0,1.310874e+09,1.310846e+09,0,NaN,0,False,t3_ir1ai,True,t3_ir1ai,/r/KamenRider/comments/ir1ai/kamen_rider_ooo_4...,1,False,True


I processed this stuff before, but I'm going to use the unprocessed body and process a different way. I only need to remove the words that could conflict with sentiment and then the model will process everything else as designed.